# Store all from category Beruf

In [4]:
#store pages from category Beruf
from wikitools import wiki, api,category
import json

def seen_wiki_categories(lang,category_page,overloop=0,n=6):
    global seen_categories_
    global seen_pages_
    where_go_categories_=[]
    #print "We are here: "+ category_page
    seen_categories_.append(category_page)
    site = wiki.Wiki("http://"+lang+".wikipedia.org/w/api.php") 
    cat = category.Category(site, category_page) # Create object for "Category:xxxxx"
    for article in cat.getAllMembersGen(titleonly=True,namespaces=[0]): # iterate through all the pages in ns 0
        if article not in seen_pages_ :
            seen_pages_.append(article)
        else:
            continue
    for subcategory in cat.getAllMembersGen(titleonly=True,namespaces=[14]):
        if subcategory not in seen_categories_ :
            where_go_categories_.append(subcategory)
        else:
            continue
    for c in where_go_categories_ :
        if c not in seen_categories_:
            if overloop<n:
                seen_wiki_categories(lang,c,overloop=overloop+1,n=n)
            else:
                print "overloop ==="+c
                pass


In [15]:

seen_categories_=[]
seen_pages_=[]
seen_wiki_categories("de","Kategorie:Beruf",overloop=0,n=3)
seen_pages_=list(set(seen_pages_))
print len(seen_pages_)
with open('de/wiki/berufe_wiki4.json', 'w') as f: 
    json.dump(seen_pages_,  f,  indent=4)

overloop ===Kategorie:Amtsbezeichnung (Deutschland)
overloop ===Kategorie:Amtsbezeichnung (Österreich)
overloop ===Kategorie:Dienstgrad
overloop ===Kategorie:Dienststellung
overloop ===Kategorie:Polizeiberuf
overloop ===Kategorie:Leiter der litauischen Zollverwaltung
overloop ===Kategorie:Mitglied des Zoll-Ski-Teams
overloop ===Kategorie:Anwaltschaft in der Schweiz
overloop ===Kategorie:Anwaltschaft in Russland
overloop ===Kategorie:Anwaltskanzlei
overloop ===Kategorie:Anwaltschaft in Deutschland
overloop ===Kategorie:Anwaltschaft in Frankreich
overloop ===Kategorie:Anwaltschaftsgeschichte
overloop ===Kategorie:Anwaltschaft in Litauen
overloop ===Kategorie:Rechtsanwalt
overloop ===Kategorie:Rechtsanwaltskammer
overloop ===Kategorie:Anwaltschaft im Vereinigten Königreich
overloop ===Kategorie:Militärisches Luftfahrtpersonal
overloop ===Kategorie:Binnenschifffahrtsberuf
overloop ===Kategorie:Seefahrtsberuf
overloop ===Kategorie:Fernsehkoch
overloop ===Kategorie:Industrieller Elektroberuf

# Levenshtein dictance

In [3]:
#compare seen_categories_ and beruf
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
words=load_simple_json('de/occupation_all.json')
wiki_pages=load_simple_json('de/wiki/berufe_wiki4.json')


both=[]
for i in words:
    if i in wiki_pages:
        both.append(i)
    if words[i][1] in wiki_pages:
        both.append(i)
print "we have ",len(both),"wiki pages from category Beruf and our list of occupations"#640 and 648(+for feminine)  

we have  702 wiki pages from category Beruf and our list of occupations


In [4]:
both2={}
for i in words:
    for j in wiki_pages:
        if i in j:
            if i not in both2.keys():
                both2[i]=[j]
            else:
                both2[i].append(j)
                #both2[i]=[a]
        #if words[i][1] in j:
        #    both2.append(i)
print len(both2)

791


In [5]:
m_no_page=load_simple_json('de/wiki/m_no_page.json')

#match no_page and wiki_berufe4
both3={}
for i in m_no_page:
    for j in wiki_pages:
        if i in j:
            if i not in both3.keys():
                both3[i]=[j]
            else:
                both3[i].append(j)
print "# of Masculine_no_pages in list of pages from category Beruf (substring inclusion):",len(both3)#18

#match wiki_berufe4 and no_page
both4={}
for i in m_no_page:
    for j in wiki_pages:
        if j in i:
            if j not in both4.keys():
                both4[j]=[i]
            else:
                both4[j].append(i)
print "# of Masculine_no_pages in list of pages from category Beruf (substring inclusion reversed):",len(both4)#205

# of Masculine_no_pages in list of pages from category Beruf (substring inclusion): 17
# of Masculine_no_pages in list of pages from category Beruf (substring inclusion reversed): 206


In [6]:
#levenshtein and string inclusion
import Levenshtein

print "With Levenshtein distance<=2 or ratio>0.8"
m_page=load_simple_json('de/wiki/m_page.json')
both6={}
k=0
m=0
for i in m_no_page:
    for j in wiki_pages: 
                    if i in j:
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                    elif Levenshtein.ratio(i, j)>0.8:
                        print "===>",i,"==",j
                        m+=1
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                    elif Levenshtein.distance(i, j)<=2:
                        k+=1
                        print i,":",j    
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
print k
print m

With Levenshtein distance<=2 or ratio>0.8
===> Planungstechniker == Zerspanungstechniker
===> Metallformer == Metallverformer
===> Farb- und Oberflächengestalter == Garten- und Grünflächengestalter
===> Gastechniker == Bautechniker
===> Gastechniker == Glasbautechniker
===> Gastechniker == Haustechniker
===> Abfallberater == Abfallbeauftragter
===> Technischer Leiter == Technischer Zeichner
===> Elektroinstallateurmeister == Elektroinstallateur
===> Betonstein- und Terrazzoherstellermeister == Betonstein- und Terrazzohersteller
===> Brandschutzglaser == Brandschutzwart
===> Brandschutzglaser == Brandschutzgutachter
===> Holzblasinstrumentenfacharbeiter == Holzblasinstrumentenmacher
===> Fachverkäufer im Lebensmittelhandwerk (Konditorei) == Fachverkäufer im Lebensmittelhandwerk
===> Fachverkäufer im Lebensmittelhandwerk (Konditorei) == Fachverkäufer im Lebensmittelhandwerk (Fleischerei)
===> Fachverkäufer im Lebensmittelhandwerk (Konditorei) == Fachverkäufer im Lebensmittelhandwerk (Bäc

In [7]:
print "======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======"
for i in both3:
    for j in both3[i]:
        if j in page_m:
            print "!!!already in pages",j
        elif j in m_links_to_feminine:
            print "!!!links to feminine",j
        elif j in m_links_to_smth:
            print "!!!links to smth",j
        elif j in m_links_to_smth.values():
            print "!!!links to smth_values",j
        elif j in page_f:
            print "!!!links to feminine",j

======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======
!!!already in pages Fachverkäufer im Lebensmittelhandwerk
!!!already in pages Fachverkäufer im Lebensmittelhandwerk (Fleischerei)
!!!already in pages Fachverkäufer im Lebensmittelhandwerk (Bäckerei)
!!!already in pages Betonstahlbieger und -flechter
!!!already in pages Zahnmedizinischer Fachassistent
!!!already in pages Fachangestellter für Bürokommunikation
!!!already in pages Fachangestellter für Bäderbetriebe
!!!already in pages Fachangestellter für Medien- und Informationsdienste
!!!already in pages Fachangestellter für Arbeitsmarktdienstleistungen
!!!already in pages Medizinischer Fachangestellter
!!!already in pages Tiermedizinischer Fachangestellter
!!!already in pages Zahnmedizinischer Fachangestellter
!!!already in pages Fachangestellter für Markt- und Sozialforschung
!!!already in pages Technischer Produktdesigner
!!!already in pages Haus- und Familienpfleger
!!!already in pages Feuerun

In [8]:

def check_pages_in_list(both6):
    new_both6={}
    for i in both6:
        if i not in new_both6.keys():
            new_both6[i]=both6[i]
        for j in both6[i]:
            if j in page_m:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],1 #1=page exist
            elif j in m_links_to_feminine:
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],2 #2=redirects to feminine
            elif j in m_links_to_smth:
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],3 #3=redirects to smth
            elif j in m_links_to_smth.values():
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],4 #4=redirects to smth value
            else:
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],0 #no page
    return new_both6
new_both6=check_pages_in_list(both6)


In [9]:
with open('de/wiki/berufe_levenshtein.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category]
    json.dump(new_both6,  f,  indent=4)
with open('de/wiki/berufe_substring.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category_withsubstring_inclusion]
    json.dump(both4,  f,  indent=4)
    


# Read all with mark 98,99 and 100 and add 99 into list of existed pages 

In [4]:
lev_corrected=load_simple_json('de/wiki/berufe_levenshtein_3105.json')

new_page_m={}
new_page_n={}
for i in lev_corrected:
    for j in lev_corrected[i]:
        if j[1]==99:
            new_page_m[i]=j[0]
            
        elif j[1]==98:
            print i,"==",j[0], "?????"
        elif j[1]==100:
            print i, "is =====>",j[0],"Neutral?"
            new_page_n[i]=j[0]
print len(new_page_m) ,   len(new_page_n)       

Audiodesigner is =====> Audiodesign Neutral?
Dekorationenmaler is =====> Dekorationsmalerei Neutral?
Medientechniker is =====> Medientechnik Neutral?
Projektassistent is =====> Projektassistenz Neutral?
Messebauer is =====> Messebau Neutral?
Filmgeschäftsführer is =====> Filmgeschäftsführung Neutral?
Pflegedienstleiter is =====> Pflegedienstleitung Neutral?
30 7


In [64]:
new_page_m_feminine_form={}
for i in new_page_m:
    if len(new_page_m[i].split(" "))==1:
        if new_page_m[i][-4:]!="mann":
            new_page_m_feminine_form[i]=new_page_m[i]+"in"
        else:
            new_page_m_feminine_form[i]=new_page_m[i].replace("mann","frau")
    else:
        if ("Verwaltungsbetriebswirt"not in new_page_m[i])&("Fachberater"not in new_page_m[i]):
            new_page_m_feminine_form[i]=new_page_m[i].replace("er ","e ")+"in"
        else:
            m=new_page_m[i].split(" ")
            new_page_m_feminine_form[i]=new_page_m[i].replace(m[0],m[0]+"in ")
new_page_m_feminine_form #check this feminine pages in wiki?


{u'Automechatroniker': u'Automobil-Mechatronikerin',
 u'Bezirksleiter': u'Bezirksamtsleiterin',
 u'B\xfcchsenmachermeister': u'B\xfcchsenmeisterin',
 u'Chemisch-technischer Assistent': u'Chemisch-Technische Assistentin',
 u'Einrichtungsfachberater': u'Einrichtungsberaterin',
 u'Fachberater im Finanzdienstleistungen': u'Fachberaterin  f\xfcr Finanzdienstleistungen',
 u'Film-Regisseur': u'Filmregisseurin',
 u'Foto- und medientechnischer Assistent': u'Gestaltungs- und medientechnische Assistentin',
 u'Gartenbauwerker': u'Gartenbaufachwerkerin',
 u'Handelslehrer': u'Diplom-Handelslehrerin',
 u'IT-System-Kaufmann': u'IT-Systemkauffrau',
 u'Kalkulator': u'Calculatorin',
 u'Kettengoldschmied': u'Kettenschmiedin',
 u'Kommunikationstechnischer Assistent': u'Informationstechnische Assistentin',
 u'Kunststoff-Formgeber': u'Kunststoffformgeberin',
 u'Logistikmeister': u'Gepr\xfcfte Logistikmeisterin',
 u'Maschinentechniker': u'Maschinenbautechnikerin',
 u'Mathematisch-technischer Assistent': u'Mat

In [65]:
with open('de/wiki/m_page_levenshtein.json', 'w') as f:
    json.dump(new_page_m,  f,  indent=4)
with open('de/wiki/n_page_levenshtein.json', 'w') as f: 
    json.dump(new_page_n,  f,  indent=4)
with open('de/wiki/m_page_levenshtein_feminine_form.json', 'w') as f: 
    json.dump(new_page_m_feminine_form,  f,  indent=4)
    

# Check if page for feminine form exist

In [71]:
from bs4 import BeautifulSoup
from wikitools import wiki, api
import json
import re


def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

    
def check_revision(revisions):
    redir=""
    page=False
    for j in revisions:
        redir_marker=["#WEITERLEITUNG","#REDIRECT","#redirect","#weiterleitung","#Weiterleitung","#Redirect"]
        if ("#WEITERLEITUNG" in j["*"])|("#REDIRECT" in j["*"])|("#redirect" in j["*"])|("#weiterleitung" in j["*"])\
        |("#Redirect" in j["*"])|("#Weiterleitung" in j["*"]):
            m = re.search(r"\[\[(.+)\]\]", j["*"])
            s = m.group(0)[2:].split('#', 1)[0].replace("]]",'')
            redir=s
        else:
            page=True
    return redir,page


    
new_page_m_feminine_form=load_simple_json('de/wiki/m_page_levenshtein_feminine_form.json')

redirection_f_new={}
page_f_new={}
no_page_f_new=[]

for i in new_page_m_feminine_form:
    beruf_f=new_page_m_feminine_form[i]
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    #last revision
    params={'action':'query','titles':beruf_f,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    
    if result["query"]["pages"].values()[0].get("revisions"):
        redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
        if redir:
            redirection_f_new[i]=redir
        if page:
            page_f_new[i]=beruf_f        
    else:

        no_page_f_new.append(beruf_f)

print page_f_new
print redirection_f_new

#check redirections
new_page_m=load_simple_json('de/wiki/m_page_levenshtein.json')
f_links_to_mascuilne_lev={}
for i in redirection_f_new:
    if redirection_f_new[i]==new_page_m[i]:
        f_links_to_mascuilne_lev[new_page_m_feminine_form[i]]=redirection_f_new[i]
    else:
        print i
with open('de/wiki/f_links_to_mascuilne_lev.json', 'w') as f: 
    json.dump(f_links_to_mascuilne_lev,  f,  indent=4)

{}
{u'Film-Regisseur': u'Filmregisseur', u'Video-Journalist': u'Videojournalist', u'IT-System-Kaufmann': u'IT-Systemkaufmann', u'Mathematisch-technischer Assistent': u'Mathematisch-Technischer Assistent'}


# Table

In [2]:
from prettytable import PrettyTable

import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
words_revert={words[i][1]:i for i in words}#feminine:masculine
#neutral=load_simple_json('de/neutral_new.json')

no_page_m=load_simple_json('de/wiki/m_no_page.json')
no_page_f=load_simple_json('de/wiki/f_no_page.json')
#no_page_n=load_simple_json('de/wiki/n_no_page.json')
page_m=load_simple_json('de/wiki/m_page.json')
page_f=load_simple_json('de/wiki/f_page.json')
#page_n=load_simple_json('de/wiki/n_page.json')
m_page_levenshtein=load_simple_json('de/wiki/m_page_levenshtein.json')
#n_page_levenshtein=load_simple_json('de/wiki/n_page_levenshtein.json')


page_f_validated=load_simple_json('de/wiki/f_page_validated.json')
page_f_ambigious=load_simple_json('de/wiki/f_page_ambigious.json')
page_f_other=load_simple_json('de/wiki/f_page_other.json')  
    
page_m_validated=load_simple_json('de/wiki/m_page_validated.json') 
page_m_ambigious=load_simple_json('de/wiki/m_page_ambigious.json')
page_m_other=load_simple_json('de/wiki/m_page_other.json')

#page_n_validated=load_simple_json('de/wiki/n_page_validated.json')
#page_n_ambigious=load_simple_json('de/wiki/n_page_ambigious.json')
#page_n_other=load_simple_json('de/wiki/n_page_other.json') 

m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
f_links_to_mascuilne=load_simple_json('de/wiki/f_links_to_mascuilne.json')
f_links_to_smth=load_simple_json('de/wiki/f_links_to_smth.json')
#redirection_n=load_simple_json('de/wiki/n_redirects.json')
f_links_to_mascuilne_lev=load_simple_json('de/wiki/f_links_to_mascuilne_lev.json')

t = PrettyTable(['-de-',"Occup.", 'Wiki pages', "Validated pages","Ambigious pages","Redirects",
                 "Redir. to oposit gender"])
t.add_row(['Masculine',len(words), len(page_m)+len(m_page_levenshtein), len(page_m_validated), len(page_m_ambigious),
           len(m_links_to_feminine)+len(m_links_to_smth), len(m_links_to_feminine)])
t.add_row(['Feminine',len(words_revert), len(page_f), len(page_f_validated), len(page_f_ambigious),
           len(f_links_to_mascuilne)+len(f_links_to_smth)+len(f_links_to_mascuilne_lev),
           len(f_links_to_mascuilne)+len(f_links_to_mascuilne_lev)])
print t

+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+
|    -de-   | Occup. | Wiki pages | Validated pages | Ambigious pages | Redirects | Redir. to oposit gender |
+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+
| Masculine |  4164  |    900     |       715       |       114       |    486    |            4            |
|  Feminine |  4164  |     12     |        7        |        3        |    276    |           185           |
+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+


In [3]:
t2 = PrettyTable(['-de-',"Occup.", 'Wiki pages', "Redirects", "No page"])
t2.add_row(['Masculine',len(words), len(page_m)+len(m_page_levenshtein), len(m_links_to_feminine)+len(m_links_to_smth),
            len(no_page_m)-len(m_page_levenshtein)])
t2.add_row(['Feminine',len(words_revert), len(page_f),
            len(f_links_to_mascuilne)+len(f_links_to_smth)+len(f_links_to_mascuilne_lev),
            len(no_page_f)-len(f_links_to_mascuilne_lev)])
print t2
print
t3 = PrettyTable(['-de-','Wiki pages', "Validated pages","Ambigious pages","Not validated pages"])
t3.add_row(['Masculine', len(page_m)+len(m_page_levenshtein), len(page_m_validated), len(page_m_ambigious),len(page_m_other)])
t3.add_row(['Feminine', len(page_f), len(page_f_validated), len(page_f_ambigious),len(page_f_other)])
print t3
print
t4 = PrettyTable(['--de--',"Redirects","Redirects to oposit gender", "Other redirects"])
t4.add_row(['Masculine',len(m_links_to_feminine)+len(m_links_to_smth),
            len(m_links_to_feminine),len(m_links_to_smth)])
t4.add_row(['Feminine',len(f_links_to_mascuilne)+len(f_links_to_smth)+len(f_links_to_mascuilne_lev),
            len(f_links_to_mascuilne)+len(f_links_to_mascuilne_lev),len(f_links_to_smth)])
print t4


+-----------+--------+------------+-----------+---------+
|    -de-   | Occup. | Wiki pages | Redirects | No page |
+-----------+--------+------------+-----------+---------+
| Masculine |  4164  |    900     |    486    |   2778  |
|  Feminine |  4164  |     12     |    276    |   3876  |
+-----------+--------+------------+-----------+---------+

+-----------+------------+-----------------+-----------------+---------------------+
|    -de-   | Wiki pages | Validated pages | Ambigious pages | Not validated pages |
+-----------+------------+-----------------+-----------------+---------------------+
| Masculine |    900     |       715       |       114       |          41         |
|  Feminine |     12     |        7        |        3        |          2          |
+-----------+------------+-----------------+-----------------+---------------------+

+-----------+-----------+----------------------------+-----------------+
|   --de--  | Redirects | Redirects to oposit gender | Other redire

In [ ]:
#add together levenshtein, manually retrieved and validated pages 
#for masculine
#for feminine
#for neutral

#what to do with neutral


#what to do with no evidence


